In [61]:
# activate line execution
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# plotly
import plotly.express as px  # (version 4.7.0 or higher)
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import numpy as np

In [62]:
# !pip install wandb -qqq
import os
os.environ["WANDB_API_KEY"] = "0f780ac8a470afe6cb7fc474ff3794772c660465"

os.environ["WANDB_NOTEBOOK_NAME"] = "jup_res"

import wandb
# wandb.login()
api = wandb.Api()





# selected_tag = 'Pred'

runs = api.runs("hokarami/TEEDAM_supervised")


In [63]:
import pandas as pd 


# Project is specified by <entity/project-name>
def dl_runs(all_runs, selected_tag=None):




    summary_list, config_list, name_list, path_list = [], [], [], []
    for run in all_runs: 

        if (selected_tag not in run.tags) and (selected_tag is not None):
            continue
        # .summary contains the output keys/values for metrics like accuracy.
        #  We call ._json_dict to omit large files 
        summary_list.append(run.summary._json_dict)

        # .config contains the hyperparameters.
        #  We remove special values that start with _.
        

        config_list.append(
            {k: v for k,v in run.config.items()
            if not k.startswith('_')})

        # .name is the human-readable name of the run.
        name_list.append(run.name)
        path_list.append(run.path)

    runs_df = pd.DataFrame({
        "summary": summary_list,
        "config": config_list,
        "name": name_list,
        "path": path_list

        })

    # runs_df.to_csv("project.csv")
    return runs_df

In [64]:
# q = runs_df['name'].str.contains('\[R2-')
# runs_df = runs_df[q]


runs_df = dl_runs(runs, selected_tag='RD74-table')

runs_df
runs_df.iloc[0].summary.keys()
runs_df.iloc[0].config.keys()

,summary,config,name,path


IndexError: single positional indexer is out-of-bounds

# Table I
Effect of time concatenation and single+mark

In [59]:
runs = api.runs("hokarami/TEEDAM_unsupervised")
df_filt = dl_runs(runs, selected_tag= "TL70")

In [60]:
df_filt

,summary,config,name,path


In [5]:
runs = api.runs("hokarami/TEEDAM_unsupervised_timeCat")
df_filt = dl_runs(runs, selected_tag='H70')
df_filt


,summary,config,name,path
0,"{'Valid-NextType(MC)/acc': 0.5067098105126473,...","{'w': [1, 1, 1], 'lr': 0.003, 'log': 'log.txt'...",[H70-TE__pp_mc-sum-d1]1888307-,"[hokarami, TEEDAM_unsupervised_timeCat, swclgn94]"
1,"{'Test-NextType(ML)/f1-weighted-CIF': 0, 'Vali...",{'w': '[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1....,[H70-TE__pp_ml-concat-d1]1846111-,"[hokarami, TEEDAM_unsupervised_timeCat, ngompq5p]"
2,{'Test-NextType(ML)/f1-weighted': 0.1698214655...,{'w': '[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1....,[H70-TE__pp_single_mark-sum-d1]1847764-,"[hokarami, TEEDAM_unsupervised_timeCat, usls7sls]"
3,"{'Test-NextTime/RMSE': 3.4010276304092084, 'ti...",{'w': '[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1....,[H70-TE__pp_single_mark-concat-d1]1848546-,"[hokarami, TEEDAM_unsupervised_timeCat, rq60tih1]"
4,{'Best-Test-CIF/LL-#events': -4.82887770046604...,{'w': '[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1....,[H70-TE__pp_ml-sum-d1]1843295-,"[hokarami, TEEDAM_unsupervised_timeCat, mtuvvfln]"
...,...,...,...,...
374,{'Valid-NextType(ML)/precision-weighted': 0.34...,{'w': '[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1....,[H70-TE__nextmark-concat-d-2]1785157-,"[hokarami, TEEDAM_unsupervised_timeCat, j9z1o2jy]"
375,{'Test-NextType(MC)/f1-macro': 0.4492456530605...,"{'w': [1, 1, 1], 'lr': 0.003, 'log': 'log.txt'...",[H70-TE__nextmark-concat-d-2]1787979-,"[hokarami, TEEDAM_unsupervised_timeCat, imm1g4my]"
376,{'Valid-NextType(ML)/precision-weighted': 0.28...,{'w': '[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1....,[H70-TE__nextmark-concat-d-2]1787649-,"[hokarami, TEEDAM_unsupervised_timeCat, me3gzd7s]"
377,{'Test-NextTime/rmse_norm': 293.05339260175583...,"{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[H70-TE__nextmark-concat-d-2]1786183-,"[hokarami, TEEDAM_unsupervised_timeCat, i0kftsh7]"


In [45]:
i_groupby_row =['dataset']
i_groupby_col = ['time_enc']
i_groupby_col=['mod','diag_offset','time_enc']
i_agg = ['split']
i_metric = ['Best-Test-CIF/LL-#events', 'Best-Test-NextType(ML)/auc-ovo-weighted','Best-Test-NextType(MC)/f1-weighted']
dict_rename_metrics = {'Best-Test-CIF/LL-#events':'LL/#events', 'Best-Test-NextType(ML)/auc-ovo-weighted':'AUROC','Best-Test-NextType(MC)/f1-weighted':'F1-Score'}
# i_metric = ['Test-CIF/NLL-#events']
i_stats=['mean','std']
# i_stats=['mean']

#df_filt.iloc[0].config.keys()

d_name = [x for x in df_filt.name]

df_config = pd.DataFrame(   [{k:v for k,v in x.items()} for x in df_filt.config]    )

df_summary = pd.DataFrame(   [{k:v for k,v in x.items()} for x in df_filt.summary]    )

df = pd.concat([df_config, df_summary],axis=1)

# df.name = df_filt.name
df = df[i_groupby_row + i_groupby_col + i_agg + i_metric]

df = df.rename(columns=dict_rename_metrics)
i_metric = list(dict_rename_metrics.values())

# df = df.melt(id_vars=i_groupby_row + i_groupby_col + i_agg ,value_vars=i_metric, var_name='Metric')




a0 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].size().rename('runs')


a1 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].aggregate(i_stats).round(3)

a1 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].aggregate(lambda x:f'{round(x.mean()*1,2):.2f}({round(x.std()*1,2):.2f})'  ).reset_index() #  

# a4 = df.groupby(i_by)[i_metric].apply(lambda x:x.round(3).tolist())   # x.round(3).tolist()

a1.loc[   a1['mod'].isin(['ml','mc']) , 'mod']='PP(ML/MC)'
a1.loc[   a1['mod'].isin(['none']) , 'mod']='AE'
a1.loc[   a1['mod'].isin(['single']) , 'mod']='PP(single)'



df_result = a1.melt(id_vars=i_groupby_row + i_groupby_col,value_vars=i_metric, var_name='Metric')



# df = df.melt(id_vars=i_groupby_row + i_groupby_col + i_agg ,value_vars=i_metric, var_name='Metric')

df_result = df_result.pivot(index=i_groupby_row+['Metric'],columns=i_groupby_col)

# remove irrelevant metrics
df_result = df_result.replace('nan (nan)', np.nan).dropna(axis=0,how='all')
df_result


value                                         \
mod                        PP(ML/MC)                                          
diag_offset                       -2                        -1                
time_enc                      concat          sum       concat          sum   
dataset      Metric                                                           
Retweets(MC) AUROC          nan(nan)     nan(nan)     nan(nan)     nan(nan)   
             F1-Score      0.53(nan)    0.35(nan)    0.52(nan)    0.35(nan)   
             LL/#events  -10.28(nan)  -22.16(nan)  -12.02(nan)  -18.85(nan)   
Retweets(ML) AUROC        0.74(0.01)   0.72(0.01)   0.74(0.01)   0.72(0.01)   
             F1-Score       nan(nan)     nan(nan)     nan(nan)     nan(nan)   
             LL/#events  -1.34(0.03)  -1.58(0.03)  -1.50(0.05)  -1.58(0.03)   
SO           AUROC          nan(nan)     nan(nan)     nan(nan)     nan(nan)   
             F1-Score     0.33(0.01)   0.33(0.01)   0.33(0.01)   0.32(0.00)   
             LL/#events  -2.02(0.02)  -2.02(0.02)  -2.02(0.01)  -2.04(0.01)   
Synthea      AUROC        0.58(0.02)   0.59(0.05)   0.60(0.02)   0.61(0.04)   
             F1-Score       nan(nan)     nan(nan)     nan(nan)     nan(nan)   
             LL/#events  -5.49(0.49)  -5.43(0.48)  -5.54(0.46)  -5.35(0.41)   

                                                                             \
mod                                                                           
diag_offset                        0                         1                
time_enc                      concat          sum       concat          sum   
dataset      Metric                                                           
Retweets(MC) AUROC          nan(nan)     nan(nan)     nan(nan)     nan(nan)   
             F1-Score      0.53(nan)    0.45(nan)    0.44(nan)    0.44(nan)   
             LL/#events  -16.29(nan)  -13.22(nan)  -12.03(nan)  -21.68(nan)   
Retweets(ML) AUROC        0.74(0.02)   0.71(0.02)   0.66(0.03)   0.66(0.03)   
             F1-Score       nan(nan)     nan(nan)     nan(nan)     nan(nan)   
             LL/#events  -1.49(0.03)  -1.60(0.04)  -1.67(0.04)  -1.68(0.05)   
SO           AUROC          nan(nan)     nan(nan)     nan(nan)     nan(nan)   
             F1-Score     0.32(0.00)   0.32(0.00)   0.32(0.01)   0.32(0.00)   
             LL/#events  -2.04(0.01)  -2.05(0.01)  -2.08(0.05)  -2.07(0.02)   
Synthea      AUROC        0.60(0.05)   0.60(0.05)   0.57(0.06)   0.61(0.05)   
             F1-Score       nan(nan)     nan(nan)     nan(nan)     nan(nan)   
             LL/#events  -5.50(0.46)  -5.33(0.42)  -5.61(0.41)  -5.26(0.39)   

                                                 ...                          \
mod                              AE              ...                           
diag_offset                      -2              ...           1               
time_enc                     concat         sum  ...      concat         sum   
dataset      Metric                              ...                           
Retweets(MC) AUROC         nan(nan)    nan(nan)  ...    nan(nan)    nan(nan)   
             F1-Score     0.63(nan)   0.62(nan)  ...   0.59(nan)   0.59(nan)   
             LL/#events    nan(nan)    nan(nan)  ...    nan(nan)    nan(nan)   
Retweets(ML) AUROC       0.71(0.02)  0.69(0.03)  ...  0.66(0.03)  0.66(0.03)   
             F1-Score      nan(nan)    nan(nan)  ...    nan(nan)    nan(nan)   
             LL/#events    nan(nan)    nan(nan)  ...    nan(nan)    nan(nan)   
SO           AUROC         nan(nan)    nan(nan)  ...    nan(nan)    nan(nan)   
             F1-Score    0.38(0.01)  0.36(0.01)  ...  0.32(0.01)  0.33(0.00)   
             LL/#events    nan(nan)    nan(nan)  ...    nan(nan)    nan(nan)   
Synthea      AUROC       0.90(0.00)  0.89(0.01)  ...  0.84(0.05)  0.87(0.03)   
             F1-Score      nan(nan)    nan(nan)  ...    nan(nan)    nan(nan)   
             LL/#events    nan(nan)    nan(nan)  ...    nan(nan

In [59]:
df_result2 = df_result.transpose().sort_values(by=i_groupby_col,axis=0)

# df_result2

df_result2 = df_result2.replace('nan(nan)', np.nan).replace(r'\(nan\)$', '',regex=True).dropna(how='all', axis=1).replace(np.nan, '')

df_result2

dataset                               Retweets(MC)            Retweets(ML)  \
Metric                                    F1-Score LL/#events        AUROC   
      mod        diag_offset time_enc                                        
value AE         -2          concat           0.63              0.71(0.02)   
                             sum              0.62              0.69(0.03)   
                 -1          concat           0.63              0.71(0.02)   
                             sum              0.62              0.69(0.02)   
                  0          concat           0.63              0.70(0.03)   
                             sum              0.61              0.69(0.03)   
                  1          concat           0.59              0.66(0.03)   
                             sum              0.59              0.66(0.03)   
      PP(ML/MC)  -2          concat           0.53     -10.28   0.74(0.01)   
                             sum              0.35     -22.16   0.72(0.01)   
                 -1          concat           0.52     -12.02   0.74(0.01)   
                             sum              0.35     -18.85   0.72(0.01)   
                  0          concat           0.53     -16.29   0.74(0.02)   
                             sum              0.45     -13.22   0.71(0.02)   
                  1          concat           0.44     -12.03   0.66(0.03)   
                             sum              0.44     -21.68   0.66(0.03)   
      PP(single) -2          concat           0.57      -6.79   0.71(0.02)   
                             sum              0.56      -7.96   0.68(0.04)   
                 -1          concat           0.53      -6.64   0.70(0.03)   
                             sum              0.54     -10.34   0.68(0.03)   
                  0          concat           0.57      -6.58   0.70(0.03)   
                             sum              0.54     -10.03   0.68(0.03)   
                  1          concat           0.56      -6.44   0.66(0.03)   
                             sum              0.32      -9.02   0.66(0.03)   

dataset                                                     SO               \
Metric                                  LL/#events    F1-Score   LL/#events   
      mod        diag_offset time_enc                                         
value AE         -2          concat                 0.38(0.01)                
                             sum                    0.36(0.01)                
                 -1          concat                 0.39(0.01)                
                             sum                    0.35(0.02)                
                  0          concat                 0.38(0.02)                
                             sum                    0.34(0.01)                
                  1          concat                 0.32(0.01)                
                             sum                    0.33(0.00)                
      PP(ML/MC)  -2          concat    -1.34(0.03)  0.33(0.01)  -2.02(0.02)   
                             sum       -1.58(0.03)  0.33(0.01)  -2.02(0.02)   
                 -1          concat    -1.50(0.05)  0.33(0.01)  -2.02(0.01)   
                             sum       -1.58(0.03)  0.32(0.00)  -2.04(0.01)   
                  0          concat    -1.49(0.03)  0.32(0.00)  -2.04(0.01)   
                             sum       -1.60(0.04)  0.32(0.00)  -2.05(0.01)   
                  1          concat    -1.67(0.04)  0.32(0.01)  -2.08(0.05)   
                             sum       -1.68(0.05)  0.32(0.00)  -2.07(0.02)   
      PP(single) -2          concat     1.99(0.06)  0.37(0.01)  -0.51(0.01)   
                             sum        1.86(0.09)  0.35(0.00)  -0.54(0.01)   
                 -1          concat     2.02(0.05)  0.38(0.01)  -0.50(0.01)   
                             sum        1.87(0.10)  0.35(0.01)  -0.54(0.01)   
                  0          concat     2.08(0.05)  0.36(0.01)  -0.53(0.01) 

In [28]:
a1 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].aggregate('mean').round(3).reset_index()


a1.loc[   a1['mod'].isin(['ml','mc']) , 'mod']='PP(ML/MC)'
a1.loc[   a1['mod'].isin(['none']) , 'mod']='AE'
a1.loc[   a1['mod'].isin(['single']) , 'mod']='PP(single)'

a1

q = a1['mod']=='PP(ML/MC)'

a1.loc[q,'LL/#events'].corr(a1.loc[q,'LL/#events'])

,dataset,mod,diag_offset,time_enc,LL/#events,AUROC,F1-Score
0,Retweets(MC),PP(ML/MC),-2,concat,-10.283,NaN,0.532
1,Retweets(MC),PP(ML/MC),-2,sum,-22.161,NaN,0.353
2,Retweets(MC),PP(ML/MC),-1,concat,-12.020,NaN,0.522
3,Retweets(MC),PP(ML/MC),-1,sum,-18.847,NaN,0.354
4,Retweets(MC),PP(ML/MC),0,concat,-16.285,NaN,0.527
...,...,...,...,...,...,...,...
91,Synthea,PP(single),-1,sum,-2.064,0.643,NaN
92,Synthea,PP(single),0,concat,-1.971,0.643,NaN
93,Synthea,PP(single),0,sum,-2.104,0.637,NaN
94,Synthea,PP(single),1,concat,-1.970,0.617,NaN


1.0

# Table II

In [43]:
runs = api.runs("hokarami/TEEDAM_unsupervised")
df_filt = dl_runs(runs, selected_tag='H70')
df_filt

,summary,config,name,path
0,"{'Valid-pred_label/loss': 0, 'Best-Test-NextTi...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[H70HH-TEnoise__pp_single_mark-concat-d0]2118955,"[hokarami, TEEDAM_unsupervised, 6da0tdoz]"
1,"{'Test-CIF/#events': 36888, 'Test-NextTime/msa...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[H70HH-TEDA__pp_single_mark-concat-d0]2113072,"[hokarami, TEEDAM_unsupervised, vkl7fm3s]"
2,{'Test-NextType(ML)/auc-ovo-weighted': 0.70602...,"{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[H70HH-TE__pp_single_mark-concat-d0]2113108,"[hokarami, TEEDAM_unsupervised, toqd29m3]"
3,{'Test-NextType(ML)/f1-weighted': 0.3194789836...,"{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[H70HH-TE__pp_single_mark-concat-d0]2112420,"[hokarami, TEEDAM_unsupervised, xpgf03nt]"
4,"{'Valid-CIF/#events': 37941, 'Best-Valid-NextT...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[H70HH-TEnoise__pp_single_mark-concat-d0]2119973,"[hokarami, TEEDAM_unsupervised, jlzppisn]"
...,...,...,...,...
160,"{'Obj': 0.3610197368421053, 'Test-pred_label/A...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[H70-TEnoise__pp_single_mark-concat-d0]1993968,"[hokarami, TEEDAM_unsupervised, 9n5cs2dt]"
161,"{'Valid-CIF/NLL': 210344.765625, 'Valid-CIF/#e...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[H70-TE__pp_single_mark-concat-d0]1999765,"[hokarami, TEEDAM_unsupervised, kbstsdnm]"
162,{'Test-NextType(ML)/precision-weighted': 0.423...,"{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[H70-TEDA__pp_single_mark-concat-d0]1993078,"[hokarami, TEEDAM_unsupervised, f3kcjkiq]"
163,"{'_wandb': {'runtime': 4128}, 'Valid-NextTime/...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[H70-TEnoise__pp_single_mark-concat-d0]1997768,"[hokarami, TEEDAM_unsupervised, yycj47wo]"


In [34]:
i_groupby_row =[]
i_groupby_col = ['time_enc']
i_groupby_col=['dataset','setting','INPUT','diag_offset']
i_agg = ['split']
i_metric = ['Best-Test-CIF/LL-#events', 'Best-Test-NextType(ML)/auc-ovo-weighted']
dict_rename_metrics = {'Best-Test-CIF/LL-#events':'LL/#events', 'Best-Test-NextType(ML)/auc-ovo-weighted':'AUROC'}
# i_metric = ['Test-CIF/NLL-#events']
i_stats=['mean','std']
# i_stats=['mean']

#df_filt.iloc[0].config.keys()

d_name = [x for x in df_filt.name]

df_config = pd.DataFrame(   [{k:v for k,v in x.items()} for x in df_filt.config]    )

df_summary = pd.DataFrame(   [{k:v for k,v in x.items()} for x in df_filt.summary]    )

df = pd.concat([df_config, df_summary],axis=1)

# df.name = df_filt.name
df = df[i_groupby_row + i_groupby_col + i_agg + i_metric]

df = df.rename(columns=dict_rename_metrics)
i_metric = list(dict_rename_metrics.values())

# df = df.melt(id_vars=i_groupby_row + i_groupby_col + i_agg ,value_vars=i_metric, var_name='Metric')




a0 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].size().rename('runs')

a1 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].aggregate(i_stats).round(3)

a1 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].aggregate(lambda x:f'{round(x.mean()*1,2):.2f}({round(x.std()*1,2):.2f})'  ).reset_index() #  


# a4 = df.groupby(i_by)[i_metric].apply(lambda x:x.round(3).tolist())   # x.round(3).tolist()

# a1.loc[   a1['mod'].isin(['ml','mc']) , 'mod']='PP(ML/MC)'
# a1.loc[   a1['mod'].isin(['none']) , 'mod']='AE'
# a1.loc[   a1['mod'].isin(['single']) , 'mod']='PP(single)'





df_result = a1.melt(id_vars=i_groupby_row + i_groupby_col,value_vars=i_metric, var_name='Metric')



# df = df.melt(id_vars=i_groupby_row + i_groupby_col + i_agg ,value_vars=i_metric, var_name='Metric')

df_result = df_result.pivot(index=i_groupby_row+['Metric'],columns=i_groupby_col)

# remove irrelevant metrics
df_result = df_result.replace('nan (nan)', np.nan).dropna(axis=0,how='all')
df_result


value                                                      \
dataset              P12                                                       
setting              mc1                                    mc2                
INPUT                 TE        TEDAM      TENOISE           TE        TEDAM   
diag_offset            0            0            0            0            0   
Metric                                                                         
AUROC         0.80(0.04)   0.84(0.02)   0.80(0.06)   0.78(0.02)   0.83(0.01)   
LL/#events   -1.34(0.04)  -1.30(0.03)  -1.35(0.05)  -1.35(0.01)  -1.32(0.02)   

                                                                              \
dataset                                                                  P19   
setting                            sc                                    mc1   
INPUT            TENOISE           TE        TEDAM      TENOISE           TE   
diag_offset            0            0            0            0            0   
Metric                                                                         
AUROC         0.82(0.01)   0.73(0.05)   0.77(0.05)   0.73(0.06)   0.78(0.05)   
LL/#events   -1.34(0.01)  -1.41(0.03)  -1.38(0.04)  -1.40(0.04)  -0.87(0.26)   

                                                                              \
dataset                                                                        
setting                                        mc2                             
INPUT              TEDAM      TENOISE           TE        TEDAM      TENOISE   
diag_offset            0            0            0            0            0   
Metric                                                                         
AUROC         0.86(0.02)   0.76(0.05)   0.61(0.05)   0.74(0.11)   0.66(0.05)   
LL/#events   -0.72(0.27)  -0.88(0.24)  -1.17(0.19)  -1.02(0.17)  -1.23(0.26)   

                                                    
dataset                                             
setting               sc                            
INPUT                 TE        TEDAM      TENOISE  
diag_offset            0            0            0  
Metric                                              
AUROC         0.78(0.04)   0.88(0.03)   0.78(0.03)  
LL/#events   -0.94(0.29)  -0.80(0.28)  -0.91(0.27)

In [35]:
df_result.transpose().sort_values(by=i_groupby_col,axis=0)



Metric                                          AUROC   LL/#events
      dataset setting INPUT   diag_offset                         
value P12     mc1     TE      0            0.80(0.04)  -1.34(0.04)
                      TEDAM   0            0.84(0.02)  -1.30(0.03)
                      TENOISE 0            0.80(0.06)  -1.35(0.05)
              mc2     TE      0            0.78(0.02)  -1.35(0.01)
                      TEDAM   0            0.83(0.01)  -1.32(0.02)
                      TENOISE 0            0.82(0.01)  -1.34(0.01)
              sc      TE      0            0.73(0.05)  -1.41(0.03)
                      TEDAM   0            0.77(0.05)  -1.38(0.04)
                      TENOISE 0            0.73(0.06)  -1.40(0.04)
      P19     mc1     TE      0            0.78(0.05)  -0.87(0.26)
                      TEDAM   0            0.86(0.02)  -0.72(0.27)
                      TENOISE 0            0.76(0.05)  -0.88(0.24)
              mc2     TE      0            0.61(0.05)  -1.17(0.19)
                      TEDAM   0            0.74(0.11)  -1.02(0.17)
                      TENOISE 0            0.66(0.05)  -1.23(0.26)
              sc      TE      0            0.78(0.04)  -0.94(0.29)
                      TEDAM   0            0.88(0.03)  -0.80(0.28)
                      TENOISE 0            0.78(0.03)  -0.91(0.27)

In [48]:
runs = api.runs("hokarami/TEEDAM_unsupervised")
df_filt = dl_runs(runs, selected_tag='H70')
df_filt

df_config = pd.DataFrame(   [{k:v for k,v in x.items()} for x in df_filt.config]    )

df_summary = pd.DataFrame(   [{k:v for k,v in x.items()} for x in df_filt.summary]    )
df_path = df_filt.path.apply(lambda x:'/'.join(x))

df = pd.concat([df_config, df_summary, df_path],axis=1)

q=(df['dataset']=='P12') & (df['setting']=='mc2') & (df['INPUT']=='TEDAM') & (df['diag_offset']==1) & (df['test_center']=='1')
# df[q]
# list(df.columns)
df.loc[q,'path']

115    hokarami/TEEDAM_unsupervised/zo2u72u2
Name: path, dtype: object

## new

In [78]:
api = wandb.Api()
runs = api.runs("hokarami/TEEDAM_unsupervised")
df_filt = dl_runs(runs, selected_tag='RD74-TableII-v2')
df_filt

,summary,config,name,path
0,"{'time-Time/train_epoch': 0.3077206532160441, ...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[RD74-R3-TEE_C1-TEDA__pp_single_mark-concat-d0...,"[hokarami, TEEDAM_unsupervised, k1oc8ys4]"
1,"{'_timestamp': 1686731132.3743474, 'Best-Valid...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[RD74-R3-TEE_C1-TEDA__pp_single_mark-concat-d0...,"[hokarami, TEEDAM_unsupervised, woa6limu]"
2,"{'Valid-CIF/NLL': 32259.703125, 'Test-NextTime...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[RD74-R3-TEE_C1-TEDA__pp_single_mark-concat-d0...,"[hokarami, TEEDAM_unsupervised, 3cxw9u1g]"
3,"{'Best-Valid-pred_label/loss': 0, 'Valid-pred_...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[RD74-R3-TEE_C1-TE__pp_single_mark-concat-d0]2...,"[hokarami, TEEDAM_unsupervised, zpno66hm]"
4,"{'Test-pred_label/AUPRC': 0.28113807538850766,...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[RD74-R3-TEE_C1-TEDA__pp_single_mark-concat-d0...,"[hokarami, TEEDAM_unsupervised, nxxmz5pj]"
5,"{'max_obj_val': 0.4018264840182649, 'Best-Test...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[RD74-R3-TEE_C1-TEDA__pp_single_mark-concat-d0...,"[hokarami, TEEDAM_unsupervised, 6gzv82mg]"
6,"{'Test-CIF/#events': 24909, 'Valid-pred_label/...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[RD74-R3-TEE_C1-TE__pp_single_mark-concat-d0]2...,"[hokarami, TEEDAM_unsupervised, gm9ivkd4]"
7,"{'Test-pred_label/AUPRC': 0.16103590401065243,...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[RD74-R3-TEE_C1-TE__pp_single_mark-concat-d0]2...,"[hokarami, TEEDAM_unsupervised, w7wlbagl]"
8,"{'LR': 1e-05, '_wandb': {'runtime': 1232}, 'Te...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[RD74-R3-TEE_C1-TE__pp_single_mark-concat-d0]2...,"[hokarami, TEEDAM_unsupervised, mt740skg]"
9,{'Valid-NextType(ML)/precision-weighted': 0.62...,"{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[RD74-R3-TEE_C1-TE__pp_single_mark-concat-d0]2...,"[hokarami, TEEDAM_unsupervised, mt8wlp3m]"


In [79]:
i_groupby_row =['dataset','INPUT']
i_groupby_col = ['time_enc']
i_groupby_col=[]
i_agg = ['split']
i_metric = ['Best-Test-CIF/LL-#events', 'Best-Test-NextType(ML)/auc-ovo-weighted','Best-Test-pred_label/AUPRC']
dict_rename_metrics = {'Best-Test-CIF/LL-#events':'LL/#events', 'Best-Test-NextType(ML)/auc-ovo-weighted':'AUROC','Best-Test-pred_label/AUPRC':'AUPRC'}
# i_metric = ['Test-CIF/NLL-#events']
i_stats=['mean','std']
# i_stats=['mean']

#df_filt.iloc[0].config.keys()

d_name = [x for x in df_filt.name]

df_config = pd.DataFrame(   [{k:v for k,v in x.items()} for x in df_filt.config]    )

df_summary = pd.DataFrame(   [{k:v for k,v in x.items()} for x in df_filt.summary]    )

df = pd.concat([df_config, df_summary],axis=1)

# df.name = df_filt.name
df = df[i_groupby_row + i_groupby_col + i_agg + i_metric]

df = df.rename(columns=dict_rename_metrics)
df[['AUROC','AUPRC']] = df[['AUROC','AUPRC']]*100
i_metric = list(dict_rename_metrics.values())

# df = df.melt(id_vars=i_groupby_row + i_groupby_col + i_agg ,value_vars=i_metric, var_name='Metric')




a0 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].size().rename('runs')

a1 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].aggregate(i_stats).round(3)
a1
a1 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].aggregate(lambda x:f'{round(x.mean()*1,2):.2f}({round(x.std()*1,2):.2f})'  )#.reset_index() #  
a1

# a4 = df.groupby(i_by)[i_metric].apply(lambda x:x.round(3).tolist())   # x.round(3).tolist()

# a1.loc[   a1['mod'].isin(['ml','mc']) , 'mod']='PP(ML/MC)'
# a1.loc[   a1['mod'].isin(['none']) , 'mod']='AE'
# a1.loc[   a1['mod'].isin(['single']) , 'mod']='PP(single)'





# df_result = a1.melt(id_vars=i_groupby_row + i_groupby_col,value_vars=i_metric, var_name='Metric')



# # df = df.melt(id_vars=i_groupby_row + i_groupby_col + i_agg ,value_vars=i_metric, var_name='Metric')

# df_result = df_result.pivot(index=i_groupby_row+['Metric'],columns=i_groupby_col)

# # remove irrelevant metrics
# df_result = df_result.replace('nan (nan)', np.nan).dropna(axis=0,how='all')
# df_result


LL/#events          AUROC          AUPRC       
                    mean    std    mean    std    mean    std
dataset INPUT                                                
P12     TE        -1.351  0.020  82.337  0.259  30.490  2.036
        TEDAM     -1.332  0.023  84.840  0.771  30.929  2.930
P19     TE        -0.713  0.054  90.311  0.325  19.405  2.653
        TEDAM     -0.680  0.232  89.737  2.975  27.051  6.097

LL/#events        AUROC        AUPRC
dataset INPUT                                       
P12     TE     -1.35(0.02)  82.34(0.26)  30.49(2.04)
        TEDAM  -1.33(0.02)  84.84(0.77)  30.93(2.93)
P19     TE     -0.71(0.05)  90.31(0.32)  19.41(2.65)
        TEDAM  -0.68(0.23)  89.74(2.98)  27.05(6.10)

In [67]:
df_result.transpose().sort_values(by=i_groupby_col,axis=0)


dataset                                                     P12  \
run_name [RD74-R3-TEE_C1-TEDA__pp_single_mark-concat-d0]2696574   
split                                                         0   
Metric                                               LL/#events   
value                                            -1.34(nan)       

dataset                                                          \
run_name [RD74-R3-TEE_C1-TEDA__pp_single_mark-concat-d0]2697244   
split                                                         3   
Metric                                               LL/#events   
value                                            -1.31(nan)       

dataset                                                          \
run_name [RD74-R3-TEE_C1-TEDA__pp_single_mark-concat-d0]2702592   
split                                                         4   
Metric                                               LL/#events   
value                                            -1.37(nan)       

dataset                                                          \
run_name [RD74-R3-TEE_C1-TEDA__pp_single_mark-concat-d0]2704335   
split                                                         2   
Metric                                               LL/#events   
value                                            -1.37(nan)       

dataset                                                          \
run_name [RD74-R3-TEE_C1-TEDA__pp_single_mark-concat-d0]2705925   
split                                                         1   
Metric                                               LL/#events   
value                                            -1.38(nan)       

dataset                                                          \
run_name [RD74-R3-TEE_C1-TEDA__pp_single_mark-concat-d0]2705948   
split                                                         4   
Metric                                               LL/#events   
value                                            -1.31(nan)       

dataset                                                          \
run_name [RD74-R3-TEE_C1-TEDA__pp_single_mark-concat-d0]2709270   
split                                                         2   
Metric                                               LL/#events   
value                                            -1.41(nan)       

dataset                                                        \
run_name [RD74-R3-TEE_C1-TE__pp_single_mark-concat-d0]2695680   
split                                                       0   
Metric                                             LL/#events   
value                                            -1.35(nan)     

dataset                                                        \
run_name [RD74-R3-TEE_C1-TE__pp_single_mark-concat-d0]2696978   
split                                                       2   
Metric                                             LL/#events   
value                                            -1.38(nan)     

dataset                                                        ...  \
run_name [RD74-R3-TEE_C1-TE__pp_single_mark-concat-d0]2697375  ...   
split                                                       3  ...   
Metric                                             LL/#events  ...   
value                                            -1.33(nan)    ...   

dataset                                                     P19  \
run_name [RD74-R3-TEE_C1-TEDA__pp_single_mark-concat-d0]2693896   
split                                                         1   
Metric                                                    AUPRC   
value                                             0.35(nan)       

dataset                                                          \
run_name [RD74-R3-TEE_C1-TEDA__pp_single_mark-concat-d0]2695762   
split                                                         0   
Metric                                                    AUPRC   
value                                    

# Table III

In [5]:
runs = api.runs("hokarami/TEEDAM_supervised")
# df_filt = dl_runs(runs, selected_tag='H70HHGGG') # H70PretrainedOnly
df_filt = dl_runs(runs, selected_tag='RD74-table') # H70PretrainedOnly
len(df_filt)


55

In [7]:
i_groupby_row =['dataset','test_center','setting']
i_groupby_col = ['event_enc','mod']
i_groupby_col=['INPUT','OUTPUT']
i_agg = ['split']

# 'knn-ps-mean',
i_metric = ['Best-Test-pred_label/f1-binary','Best-Test-pred_label/AUPRC','Best-Test-pred_label/AUROC']
# i_metric = ['Test-CIF/NLL-#events']
i_stats=['mean','std']
# i_stats=['mean']

#df_filt.iloc[0].config.keys()

d_name = [x for x in df_filt.name]

df_config = pd.DataFrame(   [{k:v for k,v in x.items()} for x in df_filt.config]    )

df_summary = pd.DataFrame(   [{k:v for k,v in x.items()} for x in df_filt.summary]    )

df = pd.concat([df_config, df_summary],axis=1)
# df.name = df_filt.name
df = df[i_groupby_row + i_groupby_col + i_agg + i_metric]
df




a0 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].size().rename('runs')

a1 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].aggregate(i_stats).round(3)

a1 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].aggregate(lambda x:f'{round(x.mean()*1,2):.2f}({round(x.std()*1,2):.2f})'   ) #  

# a4 = df.groupby(i_by)[i_metric].apply(lambda x:x.round(3).tolist())   # x.round(3).tolist()
a1

a1.reset_index().pivot(index=i_groupby_row,columns=i_groupby_col).replace(r'\(nan\)$', '',regex=True)



# a1.to_excel("aaa.xlsx")

,dataset,test_center,setting,INPUT,OUTPUT,split,Best-Test-pred_label/f1-binary,Best-Test-pred_label/AUPRC,Best-Test-pred_label/AUROC
0,P19,1,sc,TEDAM,none-mark-label,4,0.509960,0.585141,0.936881
1,P19,1,sc,DAM,none-mark-label,4,0.545994,0.614948,0.928929
2,P19,1,sc,TEDAM,none-mark-label,3,0.578616,0.652080,0.931283
3,P19,1,sc,DAM,none-mark-label,3,0.545181,0.604926,0.917107
4,P19,1,sc,TEDAM,none-mark-label,2,0.503106,0.676728,0.932575
...,...,...,...,...,...,...,...,...,...
117,P12,0,mc2,DAM,none-mark-label,,0.502216,0.484488,0.832434
118,P19,0,mc2,DAM,none-mark-label,,0.274078,0.633099,0.862014
119,P19,0,mc2,TEDAM,none-mark-label,,0.334844,0.467754,0.748770
120,P19,1,mc2,TEDAM,none-mark-label,,0.589313,0.478888,0.896525


Best-Test-pred_label/f1-binary  \
dataset test_center setting INPUT OUTPUT                                           
P12     0           mc1     DAM   none-mark-label                     0.52(0.04)   
                            TEDAM none-mark-label                     0.51(0.04)   
                    mc2     DAM   none-mark-label                      0.50(nan)   
                            TEDAM none-mark-label                       nan(nan)   
                    sc      DAM   none-mark-label                     0.41(0.06)   
                            TEDAM none-mark-label                     0.42(0.03)   
        1           mc1     DAM   none-mark-label                     0.55(0.04)   
                            TEDAM none-mark-label                     0.55(0.04)   
                    mc2     DAM   none-mark-label                      0.54(nan)   
                            TEDAM none-mark-label                      0.54(nan)   
                    sc      DAM   none-mark-label                     0.48(0.05)   
                            TEDAM none-mark-label                     0.42(0.04)   
        2           mc1     DAM   none-mark-label                     0.54(0.04)   
                            TEDAM none-mark-label                     0.54(0.05)   
                    mc2     DAM   none-mark-label                      0.50(nan)   
                            TEDAM none-mark-label                      0.52(nan)   
                    sc      DAM   none-mark-label                     0.46(0.06)   
                            TEDAM none-mark-label                     0.42(0.05)   
P19     0           mc1     DAM   none-mark-label                     0.58(0.03)   
                            TEDAM none-mark-label                     0.55(0.03)   
                    mc2     DAM   none-mark-label                      0.27(nan)   
                            TEDAM none-mark-label                      0.33(nan)   
                    sc      DAM   none-mark-label                     0.62(0.01)   
                            TEDAM none-mark-label                     0.62(0.01)   
        1           mc1     DAM   none-mark-label                     0.57(0.05)   
                            TEDAM none-mark-label                     0.58(0.04)   
                    mc2     DAM   none-mark-label                      0.59(nan)   
                            TEDAM none-mark-label                      0.59(nan)   
                    sc      DAM   none-mark-label                     0.54(0.01)   
                            TEDAM none-mark-label                     0.53(0.03)   

                                                  Best-Test-pred_label/AUPRC  \
dataset test_center setting INPUT OUTPUT                                       
P12     0           mc1     DAM   none-mark-label                 0.51(0.05)   
                            TEDAM none-mark-label                 0.49(0.07)   
                    mc2     DAM   none-mark-label                  0.48(nan)   
                            TEDAM none-mark-label                   nan(nan)   
                    sc      DAM   none-mark-label                 0.34(0.06)   
                            TEDAM none-mark-label                 0.35(0.04)   
        1           mc1     DAM   none-mark-label                 0.54(0.04)   
                            TEDAM none-mark-label                 0.52(0.04)   
                    mc2     DAM   none-mark-label                  0.52(nan)   
                            TEDAM none-mark-label                  0.52(nan)   
                    sc      DAM   none-mark-label                 0.41(0.06)   
                            TEDAM none-mark-label                 0.33(0.04)   
        2           mc1     DAM   none-mark-label                 0.54(0.04)   
                            TEDAM none-mark-label                 0.53(0.06)   
                    mc2     DAM   none-mark-label                  0.46(nan)   
 

Best-Test-pred_label/f1-binary                  \
INPUT                                                  DAM           TEDAM   
OUTPUT                                     none-mark-label none-mark-label   
dataset test_center setting                                                  
P12     0           mc1                         0.52(0.04)      0.51(0.04)   
                    mc2                               0.50             nan   
                    sc                          0.41(0.06)      0.42(0.03)   
        1           mc1                         0.55(0.04)      0.55(0.04)   
                    mc2                               0.54            0.54   
                    sc                          0.48(0.05)      0.42(0.04)   
        2           mc1                         0.54(0.04)      0.54(0.05)   
                    mc2                               0.50            0.52   
                    sc                          0.46(0.06)      0.42(0.05)   
P19     0           mc1                         0.58(0.03)      0.55(0.03)   
                    mc2                               0.27            0.33   
                    sc                          0.62(0.01)      0.62(0.01)   
        1           mc1                         0.57(0.05)      0.58(0.04)   
                    mc2                               0.59            0.59   
                    sc                          0.54(0.01)      0.53(0.03)   

                            Best-Test-pred_label/AUPRC                  \
INPUT                                              DAM           TEDAM   
OUTPUT                                 none-mark-label none-mark-label   
dataset test_center setting                                              
P12     0           mc1                     0.51(0.05)      0.49(0.07)   
                    mc2                           0.48             nan   
                    sc                      0.34(0.06)      0.35(0.04)   
        1           mc1                     0.54(0.04)      0.52(0.04)   
                    mc2                           0.52            0.52   
                    sc                      0.41(0.06)      0.33(0.04)   
        2           mc1                     0.54(0.04)      0.53(0.06)   
                    mc2                           0.46            0.52   
                    sc                      0.43(0.05)      0.34(0.05)   
P19     0           mc1                     0.74(0.04)      0.69(0.08)   
                    mc2                           0.63            0.47   
                    sc                      0.71(0.01)      0.72(0.02)   
        1           mc1                     0.59(0.08)      0.60(0.06)   
                    mc2                           0.47            0.48   
                    sc                      0.60(0.04)      0.62(0.05)   

                            Best-Test-pred_label/AUROC                  
INPUT                                              DAM           TEDAM  
OUTPUT                                 none-mark-label none-mark-label  
dataset test_center setting                                             
P12     0           mc1                     0.84(0.03)      0.83(0.03)  
                    mc2                           0.83             nan  
                    sc                      0.76(0.04)      0.77(0.03)  
        1           mc1                     0.85(0.01)      0.86(0.02)  
                    mc2                           0.85            0.85  
                    sc                      0.80(0.02)      0.77(0.02)  
        2           mc1                     0.85(0.02)      0.85(0.03)  
                    mc2                           0.82            0.85  
                    sc                      0.80(0.04)      0.75(0.03)  
P19     0           mc1                     0.91(0.02)      0.88(0.05)  
                    mc2                           0.86            0.75  
                    sc                     

: 

## T3 new

In [50]:
api = wandb.Api()
runs = api.runs("hokarami/TEEDAM_supervised")
# df_filt = dl_runs(runs, selected_tag='H70HHGGG') # H70PretrainedOnly
df_filt = dl_runs(runs, selected_tag='RD74-TableIII') # H70PretrainedOnly
len(df_filt)

72

In [54]:
i_groupby_row =['dataset','split','run_name']
i_groupby_col = ['event_enc','mod']
i_groupby_col=['INPUT','OUTPUT']
i_agg = ['split']

i_groupby_col=[]
i_agg = []

# 'knn-ps-mean',
i_metric = ['Best-Test-pred_label/f1-binary','Best-Test-pred_label/AUPRC','Best-Test-pred_label/AUROC',
            'Best-Valid-pred_label/f1-binary','Best-Valid-pred_label/AUPRC','Best-Valid-pred_label/AUROC']
# i_metric = ['Test-CIF/NLL-#events']
i_stats=['mean','std']
# i_stats=['mean']

#df_filt.iloc[0].config.keys()

d_name = [x for x in df_filt.name]

df_config = pd.DataFrame(   [{k:v for k,v in x.items()} for x in df_filt.config]    )

df_summary = pd.DataFrame(   [{k:v for k,v in x.items()} for x in df_filt.summary]    )

df = pd.concat([df_config, df_summary],axis=1)
# df.name = df_filt.name
df = df[i_groupby_row + i_groupby_col + i_agg + i_metric]

# remove the last 6 characters from the run_name
df['run_name'] = df['run_name'].apply(lambda x:x[:-7])


df.sort_values(by=['dataset','split'] + ['Best-Test-pred_label/AUPRC'], ascending=False)
df.sort_values(by=['dataset','run_name']).groupby(['dataset','run_name'])[i_metric].aggregate(lambda x:f'{round(x.mean()*100,2):.1f}({round(x.std()*100,2):.1f})'   )


term

a0 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].size().rename('runs')

a1 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].aggregate(i_stats).round(3)

a1 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].aggregate(lambda x:f'{round(x.mean()*1,2):.2f}({round(x.std()*1,2):.2f})'   ) #  

# a4 = df.groupby(i_by)[i_metric].apply(lambda x:x.round(3).tolist())   # x.round(3).tolist()
a1

a1.reset_index().pivot(index=i_groupby_row,columns=i_groupby_col).replace(r'\(nan\)$', '',regex=True)



# a1.to_excel("aaa.xlsx")

,dataset,split,run_name,Best-Test-pred_label/f1-binary,Best-Test-pred_label/AUPRC,Best-Test-pred_label/AUROC,Best-Valid-pred_label/f1-binary,Best-Valid-pred_label/AUPRC,Best-Valid-pred_label/AUROC
49,P19,4,[RD74-TEDA__pp_ml-concat],0.470588,0.474048,0.860270,0.498567,0.506206,0.880937
47,P19,4,[RD74-TEDA__nextmark-concat],0.485294,0.472690,0.851787,0.509934,0.497564,0.874577
46,P19,4,[RD74-TL-TEDA__pp_single_mark-concat],0.494297,0.470018,0.855518,0.511945,0.492756,0.881374
48,P19,4,[RD74-TEDA__pp_single_mark-concat],0.498024,0.469760,0.847487,0.529210,0.502296,0.876258
50,P19,4,[RD74-TEDA__none-concat],0.479452,0.467694,0.861315,0.480480,0.503415,0.879524
...,...,...,...,...,...,...,...,...,...
40,P12,0,[RD74-TEDA__none-concat],0.474453,0.528130,0.853282,0.529801,0.509457,0.858242
12,P12,0,[RD74-TL-TEDA__nextmark-concat],0.509346,0.526585,0.855189,0.534737,0.527082,0.854810
39,P12,0,[RD74-DA__base-concat],0.480734,0.524128,0.833159,0.534224,0.536811,0.849811
14,P12,0,[RD74-TL-TEDA__pp_single_mark-concat],0.473118,0.512939,0.840336,0.530303,0.531264,0.847621


Best-Test-pred_label/f1-binary  \
dataset run_name                                                               
P12     [RD74-DA__base-concat]                                     50.0(1.9)   
        [RD74-TEDA__nextmark-concat]                               50.3(2.6)   
        [RD74-TEDA__none-concat]                                   49.9(1.8)   
        [RD74-TEDA__pp_ml-concat]                                  49.1(1.0)   
        [RD74-TEDA__pp_single_mark-concat]                         52.0(2.9)   
        [RD74-TL-TEDA__nextmark-concat]                            52.0(0.7)   
        [RD74-TL-TEDA__pp_ml-concat]                               48.1(2.4)   
        [RD74-TL-TEDA__pp_single_mark-concat]                      50.6(2.2)   
P19     [RD74-DA__base-concat]                                     47.9(2.3)   
        [RD74-TEDA__nextmark-concat]                               49.7(3.6)   
        [RD74-TEDA__none-concat]                                   49.4(2.9)   
        [RD74-TEDA__pp_ml-concat]                                  50.0(5.4)   
        [RD74-TEDA__pp_single_mark-concat]                         49.9(4.1)   
        [RD74-TL-TEDA__pp_single_mark-concat]                      49.9(3.0)   

                                              Best-Test-pred_label/AUPRC  \
dataset run_name                                                           
P12     [RD74-DA__base-concat]                                 53.4(2.9)   
        [RD74-TEDA__nextmark-concat]                           52.2(5.2)   
        [RD74-TEDA__none-concat]                               52.1(2.8)   
        [RD74-TEDA__pp_ml-concat]                              50.9(2.6)   
        [RD74-TEDA__pp_single_mark-concat]                     53.8(4.2)   
        [RD74-TL-TEDA__nextmark-concat]                        54.1(3.9)   
        [RD74-TL-TEDA__pp_ml-concat]                           48.5(5.3)   
        [RD74-TL-TEDA__pp_single_mark-concat]                  52.2(2.3)   
P19     [RD74-DA__base-concat]                                 48.8(5.6)   
        [RD74-TEDA__nextmark-concat]                           50.5(4.3)   
        [RD74-TEDA__none-concat]                               49.3(4.5)   
        [RD74-TEDA__pp_ml-concat]                              50.1(6.5)   
        [RD74-TEDA__pp_single_mark-concat]                     49.8(6.2)   
        [RD74-TL-TEDA__pp_single_mark-concat]                  50.2(4.8)   

                                              Best-Test-pred_label/AUROC  \
dataset run_name                                                           
P12     [RD74-DA__base-concat]                                 85.4(1.4)   
        [RD74-TEDA__nextmark-concat]                           85.3(1.6)   
        [RD74-TEDA__none-concat]                               85.3(0.2)   
        [RD74-TEDA__pp_ml-concat]                              85.2(0.6)   
        [RD74-TEDA__pp_single_mark-concat]                     86.2(0.9)   
        [RD74-TL-TEDA__nextmark-concat]                        86.3(0.8)   
        [RD74-TL-TEDA__pp_ml-concat]                           84.6(1.3)   
        [RD74-TL-TEDA__pp_single_mark-concat]                  85.1(1.2)   
P19     [RD74-DA__base-concat]                                 87.1(2.5)   
        [RD74-TEDA__nextmark-concat]                           88.1(2.3)   
        [RD74-TEDA__none-concat]                               87.5(1.5)   
        [RD74-TEDA__pp_ml-concat]                              87.6(2.4)   
        [RD74-TEDA__pp_single_mark-concat]                     88.1(2.7)   
        [RD74-TL-TEDA__pp_single_mark-concat]                  87.8(1.4)   

                                              Best-Valid-pred_label/f1-binary  \
dataset run_name                                                                
P12     [RD74-DA__base-concat]                                      51.8(2.5)   
        [RD74-TEDA__nextmark-concat]                                52.8(3.0)   
       

NameError: name 'term' is not defined

# CIF

In [ ]:
df_filt = dl_runs(runs, selected_tag='CIF')

df_filt.iloc[0].config.keys()

In [ ]:
i_groupby_row =['dataset','setting',]
i_groupby_col = ['state']
i_agg = ['test_center','split']
# i_metric = ['Best-Test-pred_label/f1-binary','Best-Test-pred_label/AUPRC']
i_metric = ['Test-CIF/NLL-#events']


#df_filt.iloc[0].config.keys()

d_name = [x for x in df_filt.name]

df_config = pd.DataFrame(   [{k:v for k,v in x.items()} for x in df_filt.config]    )

df_summary = pd.DataFrame(   [{k:v for k,v in x.items()} for x in df_filt.summary]    )

df = pd.concat([df_config, df_summary],axis=1)#.dropna(axis=1,how='all')
# df.name = df_filt.name
df = df[i_groupby_row + i_groupby_col + i_agg + i_metric]
df





a0 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].size().rename('runs')

a1 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].aggregate(['mean','std']).round(3)

a1 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].aggregate(lambda x:str(round(x.mean(),3))  + ' (' + str(round(x.std(),3))  +')' )

# a4 = df.groupby(i_by)[i_metric].apply(lambda x:x.round(3).tolist())   # x.round(3).tolist()
a1

a1.reset_index().pivot(index=i_groupby_row,columns=i_groupby_col)



# a1.to_csv("aaa.csv")